<a href="https://colab.research.google.com/github/yuukienomoto/mahjong/blob/main/%E9%BA%BB%E9%9B%80SVR%E5%86%8D%E7%8F%BE_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

麻雀研究　SVRを用いた捨て牌の危険度
推定

バイナリデータを解凍する

XMLの解読

SVRモデルの学習

ゲーム状態を管理するクラスを作る

XMLタグに応じてゲーム状態を更新する

In [ ]:
import gzip
import xml.etree.ElementTree as ET
import glob
from google.colab import drive
import time
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random

print("--- ステップ1: Google Driveのマウント ---")
try:
    drive.mount('/content/drive')
except:
    pass

# 2. フォルダパス指定
log_folder_path = '/content/drive/MyDrive/mjlog_pf4-20_n28/'
file_pattern = log_folder_path + '*.mjlog'
log_files = glob.glob(file_pattern)

if not log_files:
    print(f"エラー: {log_folder_path} に .mjlog ファイルが見つかりません。")
else:
    print(f"全 {len(log_files)} 個のファイルが見つかりました。")

    # ★★★ 最終的なデータセット ★★★
    final_dataset = []

    # ★★★【最重要】本格的な特徴量計算関数 ★★★
    #  に基づく
    def calculate_features(game_state, player_index, tile):

        features = []

        # 1. 捨てようとしている牌の種類 [cite: 31]
        # (牌コードをそのまま特徴量とする)
        features.append(tile)

        # 2. 捨てようとしている牌が場に見えている枚数 [cite: 33]
        visible_count = 0
        for hand in game_state['hands']:
            visible_count += hand.count(tile)
        for discard_pile in game_state['discards']:
            visible_count += discard_pile.count(tile)
        features.append(visible_count)

        # 3. 相手のリーチの有無 [cite: 34]
        opponent_reach = False
        for i in range(4):
            if i != player_index and game_state['reach'][i]:
                opponent_reach = True
        features.append(1 if opponent_reach else 0)

        # 4. 捨てようとしている牌が相手の河に存在するか [cite: 36]
        in_opponent_discard = False
        for i in range(4):
            if i != player_index and tile in game_state['discards'][i]:
                in_opponent_discard = True
        features.append(1 if in_opponent_discard else 0)

        # 5. 1巡以内 (4打) に誰かが捨てているか [cite: 37]
        discarded_1_turn = False
        # 直近4回の打牌履歴 (自分も含む) をチェック
        recent_discards = game_state['history'][-4:]
        for (p, t) in recent_discards:
            if t == tile:
                discarded_1_turn = True
        features.append(1 if discarded_1_turn else 0)

        # 6. 2巡以内 (8打) に誰かが捨てているか [cite: 38]
        discarded_2_turns = False
        # 直近8回の打牌履歴 (自分も含む) をチェック
        recent_discards = game_state['history'][-8:]
        for (p, t) in recent_discards:
            if t == tile:
                discarded_2_turns = True
        features.append(1 if discarded_2_turns else 0)

        # (ここで特徴量 [cite: 32, 35] を追加実装できる)

        return features


    # --- ステップ2: 全ファイル (100個) の処理 ---
    print("\n--- ステップ2: 全ファイルの解析を開始します ---")
    start_time = time.time()

    # (テストのため、まずは最初の100ファイルだけ処理)
    files_to_process = log_files

    for i, file_path in enumerate(files_to_process):

        if (i + 1) % 20 == 0: # 20ファイルごとに進捗を表示
            print(f"  処理中... {i+1} / {len(files_to_process)} ファイル")

        try:
            with gzip.open(file_path, 'rt', encoding='utf-8') as f:
                tree = ET.parse(f)
                root = tree.getroot()

            # --- ゲーム状態変数 (Game State) ---
            game_state = {
                'hands': [[], [], [], []],
                'discards': [[], [], [], []],
                'reach': [False, False, False, False],
                'history': [] # (player, tile) の打牌履歴
            }
            last_discard_event = None

            for tag in root:
                if tag.tag == 'INIT':
                    if last_discard_event:
                        final_dataset.append(last_discard_event)

                    game_state = {
                        'hands': [[], [], [], []],
                        'discards': [[], [], [], []],
                        'reach': [False, False, False, False],
                        'history': []
                    }
                    last_discard_event = None

                    hai_strings = [tag.attrib.get(h) for h in ['hai0', 'hai1', 'hai2', 'hai3']]
                    for p_idx in range(4):
                        if hai_strings[p_idx]:
                            game_state['hands'][p_idx] = [int(s) for s in hai_strings[p_idx].split(',')]

                # ツモ
                draw_prefix = tag.tag[0]
                if draw_prefix in {'T': 0, 'U': 1, 'V': 2, 'W': 3}:
                    try:
                        player_index = {'T': 0, 'U': 1, 'V': 2, 'W': 3}[draw_prefix]
                        tile_code = int(tag.tag[1:])
                        game_state['hands'][player_index].append(tile_code)
                    except (ValueError, KeyError):
                        pass

                # 打牌
                discard_prefix = tag.tag[0]
                if discard_prefix in {'D': 0, 'E': 1, 'F': 2, 'G': 3}:
                    try:
                        if last_discard_event:
                            final_dataset.append(last_discard_event)

                        player_index = {'D': 0, 'E': 1, 'F': 2, 'G': 3}[discard_prefix]
                        tile_code = int(tag.tag[1:])

                        # ★★★ 強化された特徴量でXを計算 ★★★
                        features_X = calculate_features(
                            game_state, player_index, tile_code
                        )
                        last_discard_event = {'X': features_X, 'y': 0}

                        # ゲーム状態の更新
                        if tile_code in game_state['hands'][player_index]:
                            game_state['hands'][player_index].remove(tile_code)
                        game_state['discards'][player_index].append(tile_code)
                        game_state['history'].append((player_index, tile_code)) # 打牌履歴を記録

                    except (ValueError, KeyError):
                        pass

                # リーチ
                if tag.tag == 'REACH':
                    if tag.attrib.get('step') == '1':
                         player_index = int(tag.attrib.get('who'))
                         game_state['reach'][player_index] = True

                # 和了 (ロン)
                if tag.tag == 'AGARI':
                    winner = int(tag.attrib.get('who'))
                    loser = tag.attrib.get('fromWho')

                    if loser and winner != int(loser):
                        score = int(tag.attrib.get('ten').split(',')[1])

                        if last_discard_event:
                            last_discard_event['y'] = score
                            final_dataset.append(last_discard_event)
                            last_discard_event = None

        except Exception as e:
            print(f"  ファイル {file_path} の処理中にエラー: {e}")

    # --- ステップ2 完了 ---
    end_time = time.time()
    print("\n-------------------------")
    print(f"全 {len(files_to_process)} ファイルの解析が完了しました。")
    print(f"処理時間: {end_time - start_time:.2f} 秒")
    print(f"作成された総データセット件数: {len(final_dataset)}")

    # --- ステップ3: SVRモデルの学習 ---
    print("\n--- ステップ3: SVRモデルの学習を開始します ---")

    if not final_dataset:
        print("データセットが空です。学習を中断します。")
    else:
        # 1. サンプリング [cite: 47]
        dataset_y_nonzero = [item for item in final_dataset if item['y'] > 0]
        dataset_y_zero = [item for item in final_dataset if item['y'] == 0]

        sample_rate = 0.01
        sampled_y_zero = random.sample(
            dataset_y_zero,
            int(len(dataset_y_zero) * sample_rate)
        )
        sampled_dataset = dataset_y_nonzero + sampled_y_zero
        random.shuffle(sampled_dataset)

        print(f"  ロンされたデータ (y>0): {len(dataset_y_nonzero)} 件")
        print(f"  サンプリング後 (y=0, 1%): {len(sampled_y_zero)} 件")
        print(f"  学習に使う総データ数: {len(sampled_dataset)} 件")

        # 2. Xとyに分離
        X = np.array([item['X'] for item in sampled_dataset])
        y = np.array([item['y'] for item in sampled_dataset])

        print(f"\n特徴量Xの形状: {X.shape}") # (件数, 6) <- 6次元に増えているはず
        print(f"教師値yの形状: {y.shape}")

        # 3. スケーリング
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        print("特徴量のスケーリング完了。")

        # 4. SVRモデルの学習 [cite: 50-51]
        C_value = 2**11
        Gamma_value = 2**-8

        svr_model = SVR(kernel='rbf', C=C_value, gamma=Gamma_value)

        print("\nSVRモデルの学習を開始します...")
        start_train_time = time.time()
        svr_model.fit(X_scaled, y)
        end_train_time = time.time()
        print(f"学習が完了しました！ (学習時間: {end_train_time - start_train_time:.2f} 秒)")

        # 5. 予測テスト
        print("\n--- 予測テスト (新モデル) ---")
        test_indices = random.sample(range(len(X)), 5) # 5件ランダムに選ぶ

        test_X_scaled = X_scaled[test_indices]
        test_y_actual = y[test_indices]
        test_y_pred = svr_model.predict(test_X_scaled)

        for i in range(5):
            print(f"サンプル{i}:")
            print(f"  特徴量(X): {X[test_indices[i]]}")
            print(f"  実際の点数(y): {test_y_actual[i]}")
            print(f"  SVRが予測した危険度: {test_y_pred[i]:.2f} 点")

--- ステップ1: Google Driveのマウント ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
全 3181 個のファイルが見つかりました。

--- ステップ2: 全ファイルの解析を開始します ---
  処理中... 20 / 3181 ファイル
  処理中... 40 / 3181 ファイル
  処理中... 60 / 3181 ファイル
  処理中... 80 / 3181 ファイル
  処理中... 100 / 3181 ファイル
  処理中... 120 / 3181 ファイル
  処理中... 140 / 3181 ファイル
  処理中... 160 / 3181 ファイル
  処理中... 180 / 3181 ファイル
  処理中... 200 / 3181 ファイル
  処理中... 220 / 3181 ファイル
  処理中... 240 / 3181 ファイル
  処理中... 260 / 3181 ファイル
  処理中... 280 / 3181 ファイル
  処理中... 300 / 3181 ファイル
  処理中... 320 / 3181 ファイル
  処理中... 340 / 3181 ファイル
  処理中... 360 / 3181 ファイル
  処理中... 380 / 3181 ファイル
  処理中... 400 / 3181 ファイル
  処理中... 420 / 3181 ファイル
  処理中... 440 / 3181 ファイル
  処理中... 460 / 3181 ファイル
  処理中... 480 / 3181 ファイル
  処理中... 500 / 3181 ファイル
  処理中... 520 / 3181 ファイル
  処理中... 540 / 3181 ファイル
  処理中... 560 / 3181 ファイル
  処理中... 580 / 3181 ファイル
  処理中... 600 / 3181 ファイル
  処理中... 620 / 3181 ファイル
  処理中... 640 